# Notebook 10: Training Self-RAG Models on LegalBench

**Objective:** Train generator and critic models for Self-RAG using LegalBench training labels

**Training Components:**
1. **Generator**: Fine-tune with LoRA to generate answers with reflection tokens
2. **Critic**: Fine-tune with LoRA to predict reflection tokens (ISREL, ISSUP, ISUSE)

**Training Data:** LegalBench mini (776 queries) with labels generated by Qwen2.5-7B-Instruct

**Expected Training Time:** 2-3 hours (depends on hardware)

## 1. Setup & Dependencies

In [2]:
import sys
from pathlib import Path
import json
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from tqdm.notebook import tqdm

# ML libraries
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
)
from datasets import Dataset

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Set seeds
import random
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Setup complete")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS available: {torch.backends.mps.is_available()}")

✓ Setup complete
PyTorch version: 2.9.0
CUDA available: False
MPS available: True


## 2. Training Labels

### 2.1 Generate Training Labels

This cell generates reflection token labels for 776 LegalBench queries using Qwen2.5-7B-Instruct.

**Process:**
1. Loads queries from LegalBench-RAG benchmark
2. Retrieves relevant passages using existing FAISS index
3. Generates labels: ISREL, ISSUP, ISUSE, Retrieve tokens
4. Saves to `data/training/legalbench_training_labels.json`

**Runtime:** ~1-2 hours on first run (model downloads ~7GB if not cached)

**Note:** This cell automatically skips if labels already exist.

In [ ]:
# Generate training labels if they don't exist
LABELS_FILE = "../data/training/legalbench_training_labels.json"

if Path(LABELS_FILE).exists():
    print(f"✓ Labels already exist at {LABELS_FILE}")
    print("  Skipping generation. Delete the file to regenerate.")
else:
    print("Generating training labels...")
    print("=" * 80)
    
    # Determine device
    device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
    print(f"Using device: {device}")
    print()
    
    # Load LegalBench queries
    QUERIES_FILE = "../data/legalbench-rag/queries.json"
    with open(QUERIES_FILE, 'r') as f:
        all_queries = json.load(f)
    
    # Use first 776 queries (mini dataset)
    queries = all_queries['tests'][:776]
    print(f"Loaded {len(queries)} queries from LegalBench-RAG")
    
    # Load retriever for passage retrieval
    from retrieval.retriever import LegalRetriever
    from retrieval.embedding import EmbeddingModel
    
    INDEX_DIR = "../data/legalbench_embeddings"
    print(f"\nLoading retriever from {INDEX_DIR}...")
    
    # Initialize embedding model
    embedding_model = EmbeddingModel(model_name="sentence-transformers/all-mpnet-base-v2")
    
    # Initialize retriever
    retriever = LegalRetriever(
        embedding_model=embedding_model,
        top_k=3
    )
    
    # Load pre-built index
    retriever.load_index(INDEX_DIR)
    print("✓ Retriever loaded")
    
    # Load labeling model
    LABEL_MODEL = "Qwen/Qwen2.5-7B-Instruct"
    print(f"\nLoading labeling model: {LABEL_MODEL}...")
    
    label_tokenizer = AutoTokenizer.from_pretrained(LABEL_MODEL)
    label_model = AutoModelForCausalLM.from_pretrained(
        LABEL_MODEL,
        torch_dtype=torch.bfloat16 if device != "cpu" else torch.float32,
        device_map=device if device != "mps" else None,
    )
    
    if device == "mps":
        label_model = label_model.to("mps")
    
    print(f"✓ Model loaded on {device}")
    
    # Generate labels for each query
    training_labels = []
    
    for query_data in tqdm(queries, desc="Generating labels"):
        query = query_data['query']
        ground_truth = query_data.get('answer', '')
        dataset_source = query_data.get('dataset', 'unknown')
        
        # Retrieve passages
        try:
            results = retriever.retrieve(query, top_k=5)
            top_passage = results[0]['text'] if results else ""
        except:
            top_passage = ""
        
        # Generate reflection tokens using the labeling model
        prompt = f"""Given a legal question, retrieved passage, and answer, predict reflection tokens.

Question: {query}

Passage: {top_passage[:500]}

Answer: {ground_truth}

Predict the following tokens:
- ISREL (Is the passage relevant?): [Relevant] or [Irrelevant]
- ISSUP (Does the passage support the answer?): [Fully supported] or [Partially supported] or [No support]
- ISUSE (Should this answer be used?): [5] (best) to [1] (worst)

Reflection tokens:"""
        
        inputs = label_tokenizer(prompt, return_tensors="pt").to(label_model.device)
        
        with torch.no_grad():
            outputs = label_model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.1,
                do_sample=False,
                pad_token_id=label_tokenizer.eos_token_id,
            )
        
        response = label_tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
        
        # Parse reflection tokens from response
        reflection_tokens = {}
        
        # Simple parsing - extract tokens from response
        if "[Relevant]" in response:
            reflection_tokens['isrel'] = "[Relevant]"
        elif "[Irrelevant]" in response:
            reflection_tokens['isrel'] = "[Irrelevant]"
        
        if "[Fully supported]" in response:
            reflection_tokens['issup'] = "[Fully supported]"
        elif "[Partially supported]" in response:
            reflection_tokens['issup'] = "[Partially supported]"
        elif "[No support]" in response:
            reflection_tokens['issup'] = "[No support]"
        
        # Extract ISUSE score
        for score in ['[5]', '[4]', '[3]', '[2]', '[1]']:
            if score in response:
                reflection_tokens['isuse'] = score
                break
        
        # Always retrieve for training
        reflection_tokens['retrieve'] = "[Retrieve]"
        
        # Store training example
        training_labels.append({
            'question': query,
            'passage': top_passage,
            'answer': ground_truth,
            'reflection_tokens': reflection_tokens,
            'dataset_source': dataset_source,
        })
    
    # Save labels
    Path(LABELS_FILE).parent.mkdir(parents=True, exist_ok=True)
    with open(LABELS_FILE, 'w') as f:
        json.dump(training_labels, f, indent=2)
    
    print(f"\n✓ Generated {len(training_labels)} training labels")
    print(f"✓ Saved to {LABELS_FILE}")
    
    # Clean up memory
    del label_model, label_tokenizer
    import gc
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("\n✓ Label generation complete!")

Generating training labels...
Using device: mps

Loaded 776 queries from LegalBench-RAG

Loading retriever from ../data/legalbench_embeddings...
Loading embedding model: sentence-transformers/all-mpnet-base-v2
Model loaded on cpu
Embedding dimension: 768
Using CPU index
Created IndexFlatIP index with dimension 768
Index loaded from ../data/legalbench_embeddings/faiss_index.faiss
Total documents in index: 326783
Documents loaded from ../data/legalbench_embeddings/documents.pkl
✓ Retriever loaded

Loading labeling model: Qwen/Qwen2.5-7B-Instruct...


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✓ Model loaded on mps


Generating labels:   0%|          | 0/776 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


### 2.2 Load and Inspect Training Labels

In [ ]:
# Load training labels
LABELS_FILE = "../data/training/legalbench_training_labels.json"

if not Path(LABELS_FILE).exists():
    print(f"⚠️  Labels file not found: {LABELS_FILE}")
    print("   This should have been generated in the previous cell.")
    print("   Please run the previous cell to generate labels.")
    raise FileNotFoundError(LABELS_FILE)

with open(LABELS_FILE, 'r') as f:
    training_data = json.load(f)

print(f"Loaded {len(training_data)} training examples")
print(f"\nExample structure:")
print(json.dumps(training_data[0], indent=2)[:500] + "...")

In [ ]:
# Analyze label distribution
print("\n" + "=" * 80)
print("Label Distribution Analysis")
print("=" * 80)

token_types = ['retrieve', 'isrel', 'issup', 'isuse']

for token_type in token_types:
    if any(token_type in ex['reflection_tokens'] for ex in training_data):
        values = [ex['reflection_tokens'].get(token_type, 'N/A') for ex in training_data]
        counter = Counter(values)
        
        print(f"\n{token_type.upper()}:")
        for val, count in counter.most_common():
            if val != 'N/A':
                pct = (count / len(values)) * 100
                print(f"  {val}: {count} ({pct:.1f}%)")

# Dataset source distribution
source_counter = Counter(ex['dataset_source'] for ex in training_data)
print(f"\nDataset Sources:")
for source, count in source_counter.most_common():
    print(f"  {source}: {count}")

In [ ]:
# Visualize label distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for i, token_type in enumerate(token_types):
    values = [ex['reflection_tokens'].get(token_type, 'N/A') for ex in training_data]
    counter = Counter(v for v in values if v != 'N/A')
    
    labels = list(counter.keys())
    counts = list(counter.values())
    
    axes[i].bar(range(len(labels)), counts, color='steelblue')
    axes[i].set_xticks(range(len(labels)))
    axes[i].set_xticklabels([l[:20] for l in labels], rotation=45, ha='right')
    axes[i].set_title(f'{token_type.upper()} Distribution')
    axes[i].set_ylabel('Count')
    axes[i].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('../results/training_label_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/training_label_distribution.png")

## 3. Data Preparation

In [ ]:
# Train/validation split (80/20)
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    training_data,
    test_size=0.2,
    random_state=42,
    stratify=[ex['dataset_source'] for ex in training_data]  # Stratify by dataset
)

print(f"Training examples: {len(train_data)}")
print(f"Validation examples: {len(val_data)}")

# Verify stratification
print("\nTrain dataset distribution:")
train_sources = Counter(ex['dataset_source'] for ex in train_data)
for source, count in train_sources.most_common():
    print(f"  {source}: {count}")

print("\nValidation dataset distribution:")
val_sources = Counter(ex['dataset_source'] for ex in val_data)
for source, count in val_sources.most_common():
    print(f"  {source}: {count}")

### 3.1 Format Data for Generator Training

Generator learns to produce: `answer [ISSUP] [ISREL] [ISUSE]`

In [ ]:
def format_generator_example(example):
    """
    Format example for generator training.
    
    Input: question + passage
    Output: answer with reflection tokens
    """
    question = example['question']
    passage = example.get('passage', '')[:500]  # Limit passage length
    answer = example.get('answer', '')
    
    tokens = example['reflection_tokens']
    
    # Build input
    if passage:
        input_text = f"""Question: {question}

Passage: {passage}

Answer with reflection tokens:"""
    else:
        input_text = f"""Question: {question}

Answer with reflection tokens:"""
    
    # Build output with reflection tokens
    output_text = answer
    
    if 'issup' in tokens:
        output_text += f" {tokens['issup']}"
    if 'isrel' in tokens:
        output_text += f" {tokens['isrel']}"
    if 'isuse' in tokens:
        output_text += f" {tokens['isuse']}"
    
    return {
        'input': input_text,
        'output': output_text,
        'full_text': input_text + "\n" + output_text,
    }

# Format train and validation data
generator_train = [format_generator_example(ex) for ex in train_data]
generator_val = [format_generator_example(ex) for ex in val_data]

print("Generator Training Example:")
print("=" * 80)
print(generator_train[0]['full_text'][:500] + "...")
print("=" * 80)

### 3.2 Format Data for Critic Training

Critic learns to predict: `[ISREL] [ISSUP] [ISUSE]` given question + passage + answer

In [ ]:
def format_critic_example(example):
    """
    Format example for critic training.
    
    Input: question + passage + answer
    Output: reflection tokens
    """
    question = example['question']
    passage = example.get('passage', '')[:500]
    answer = example.get('answer', '')
    
    tokens = example['reflection_tokens']
    
    # Build input
    input_text = f"""Question: {question}

Passage: {passage}

Answer: {answer}

Predict reflection tokens:"""
    
    # Build output (only reflection tokens)
    output_tokens = []
    if 'isrel' in tokens:
        output_tokens.append(tokens['isrel'])
    if 'issup' in tokens:
        output_tokens.append(tokens['issup'])
    if 'isuse' in tokens:
        output_tokens.append(tokens['isuse'])
    
    output_text = " ".join(output_tokens)
    
    return {
        'input': input_text,
        'output': output_text,
        'full_text': input_text + "\n" + output_text,
    }

# Format train and validation data
critic_train = [format_critic_example(ex) for ex in train_data]
critic_val = [format_critic_example(ex) for ex in val_data]

print("Critic Training Example:")
print("=" * 80)
print(critic_train[0]['full_text'][:500] + "...")
print("=" * 80)

## 4. Generator Training

### 4.1 Load Base Model and Configure LoRA

In [ ]:
# Configuration
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  # Small, fast for training
GENERATOR_OUTPUT_DIR = "../models/generator_legalbench_lora"

# Create output directory
Path(GENERATOR_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print(f"Loading model: {MODEL_NAME}...")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load model
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if device != "cpu" else torch.float32,
    device_map=device if device != "mps" else None,
)

if device == "mps":
    model = model.to("mps")

print(f"✓ Model loaded on {device}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()) / 1e6:.1f}M")

In [ ]:
# Configure LoRA
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Scaling
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Attention layers
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

print("✓ LoRA configured")

### 4.2 Prepare Datasets

In [ ]:
def tokenize_function(examples, tokenizer, max_length=1024):
    """
    Tokenize examples for training.
    """
    # Tokenize
    tokenized = tokenizer(
        examples['full_text'],
        truncation=True,
        max_length=max_length,
        padding='max_length',
        return_tensors=None,
    )
    
    # Labels are same as input_ids for causal LM
    tokenized['labels'] = tokenized['input_ids'].copy()
    
    return tokenized

# Create datasets
train_dataset = Dataset.from_list(generator_train)
val_dataset = Dataset.from_list(generator_val)

# Tokenize
train_dataset = train_dataset.map(
    lambda x: tokenize_function(x, tokenizer),
    batched=False,
    desc="Tokenizing train"
)

val_dataset = val_dataset.map(
    lambda x: tokenize_function(x, tokenizer),
    batched=False,
    desc="Tokenizing validation"
)

print(f"✓ Datasets prepared")
print(f"  Train: {len(train_dataset)} examples")
print(f"  Validation: {len(val_dataset)} examples")

### 4.3 Configure Training

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=GENERATOR_OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Effective batch size = 16
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=100,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,  # Use bf16 if supported
    bf16=True if device == "cuda" else False,
    report_to="none",  # Disable wandb/tensorboard
    seed=42,
)

print("Training Configuration:")
print(f"  Epochs: {training_args.num_train_epochs}")
print(f"  Batch size: {training_args.per_device_train_batch_size} x {training_args.gradient_accumulation_steps} = {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  Learning rate: {training_args.learning_rate}")
print(f"  Warmup steps: {training_args.warmup_steps}")
print(f"  Total steps: ~{len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs}")

### 4.4 Train Generator

In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

print("Starting generator training...")
print("This will take ~1-2 hours")
print()

# Train
train_result = trainer.train()

print("\n✓ Training complete!")
print(f"  Final train loss: {train_result.metrics['train_loss']:.4f}")
print(f"  Training time: {train_result.metrics['train_runtime'] / 60:.1f} minutes")

In [ ]:
# Save final model
trainer.save_model(GENERATOR_OUTPUT_DIR)
tokenizer.save_pretrained(GENERATOR_OUTPUT_DIR)

print(f"✓ Model saved to {GENERATOR_OUTPUT_DIR}")

# Save training metrics
import pandas as pd

training_logs = pd.DataFrame(trainer.state.log_history)
training_logs.to_csv(f"{GENERATOR_OUTPUT_DIR}/training_logs.csv", index=False)
print(f"✓ Training logs saved")

### 4.5 Visualize Training Progress

In [ ]:
# Plot loss curves
logs = pd.DataFrame(trainer.state.log_history)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Training loss
train_logs = logs[logs['loss'].notna()]
ax.plot(train_logs['step'], train_logs['loss'], label='Train Loss', linewidth=2)

# Validation loss
eval_logs = logs[logs['eval_loss'].notna()]
ax.plot(eval_logs['step'], eval_logs['eval_loss'], label='Validation Loss', linewidth=2, linestyle='--')

ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Generator Training Progress')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../results/generator_training_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/generator_training_loss.png")

### 4.6 Test Generator on Validation Examples

In [ ]:
# Test on a few validation examples
print("Testing generator on validation examples...")
print("=" * 80)

model.eval()

for i in range(3):
    example = generator_val[i]
    
    # Tokenize input
    inputs = tokenizer(example['input'], return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    print(f"\nExample {i+1}:")
    print(f"Input: {example['input'][:100]}...")
    print(f"\nExpected: {example['output']}")
    print(f"\nGenerated: {generated}")
    print("-" * 80)

## 5. Critic Training

Now train the critic to predict reflection tokens.

In [ ]:
# Clear GPU memory
import gc
del model, trainer
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("Cleared memory for critic training")

### 5.1 Load Base Model and Configure LoRA

In [ ]:
# Configuration
CRITIC_OUTPUT_DIR = "../models/critic_legalbench_lora"
Path(CRITIC_OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print(f"Loading model for critic: {MODEL_NAME}...")

# Load fresh model
critic_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.bfloat16 if device != "cpu" else torch.float32,
    device_map=device if device != "mps" else None,
)

if device == "mps":
    critic_model = critic_model.to("mps")

# Apply LoRA
critic_lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

critic_model = get_peft_model(critic_model, critic_lora_config)
critic_model.print_trainable_parameters()

print("✓ Critic model configured")

### 5.2 Prepare Critic Datasets

In [ ]:
# Create datasets
critic_train_dataset = Dataset.from_list(critic_train)
critic_val_dataset = Dataset.from_list(critic_val)

# Tokenize
critic_train_dataset = critic_train_dataset.map(
    lambda x: tokenize_function(x, tokenizer, max_length=512),
    batched=False,
    desc="Tokenizing critic train"
)

critic_val_dataset = critic_val_dataset.map(
    lambda x: tokenize_function(x, tokenizer, max_length=512),
    batched=False,
    desc="Tokenizing critic validation"
)

print(f"✓ Critic datasets prepared")
print(f"  Train: {len(critic_train_dataset)} examples")
print(f"  Validation: {len(critic_val_dataset)} examples")

### 5.3 Train Critic

In [ ]:
# Training arguments for critic
critic_training_args = TrainingArguments(
    output_dir=CRITIC_OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=50,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=100,
    save_strategy="steps",
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,
    bf16=True if device == "cuda" else False,
    report_to="none",
    seed=42,
)

# Initialize trainer
critic_trainer = Trainer(
    model=critic_model,
    args=critic_training_args,
    train_dataset=critic_train_dataset,
    eval_dataset=critic_val_dataset,
    tokenizer=tokenizer,
)

print("Starting critic training...")
print("This will take ~30-60 minutes")
print()

# Train
critic_train_result = critic_trainer.train()

print("\n✓ Critic training complete!")
print(f"  Final train loss: {critic_train_result.metrics['train_loss']:.4f}")
print(f"  Training time: {critic_train_result.metrics['train_runtime'] / 60:.1f} minutes")

In [ ]:
# Save critic model
critic_trainer.save_model(CRITIC_OUTPUT_DIR)
tokenizer.save_pretrained(CRITIC_OUTPUT_DIR)

print(f"✓ Critic model saved to {CRITIC_OUTPUT_DIR}")

# Save training logs
critic_logs = pd.DataFrame(critic_trainer.state.log_history)
critic_logs.to_csv(f"{CRITIC_OUTPUT_DIR}/training_logs.csv", index=False)
print(f"✓ Critic training logs saved")

### 5.4 Visualize Critic Training

In [ ]:
# Plot critic loss curves
critic_logs_df = pd.DataFrame(critic_trainer.state.log_history)

fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# Training loss
train_logs = critic_logs_df[critic_logs_df['loss'].notna()]
ax.plot(train_logs['step'], train_logs['loss'], label='Train Loss', linewidth=2)

# Validation loss
eval_logs = critic_logs_df[critic_logs_df['eval_loss'].notna()]
ax.plot(eval_logs['step'], eval_logs['eval_loss'], label='Validation Loss', linewidth=2, linestyle='--')

ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Critic Training Progress')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../results/critic_training_loss.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Saved to results/critic_training_loss.png")

### 5.5 Test Critic

In [ ]:
# Test critic on validation examples
print("Testing critic on validation examples...")
print("=" * 80)

critic_model.eval()

for i in range(3):
    example = critic_val[i]
    
    # Tokenize input
    inputs = tokenizer(example['input'], return_tensors="pt").to(critic_model.device)
    
    # Generate
    with torch.no_grad():
        outputs = critic_model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    print(f"\nExample {i+1}:")
    print(f"Expected tokens: {example['output']}")
    print(f"Predicted tokens: {generated}")
    print("-" * 80)

## 6. Summary & Next Steps

In [ ]:
# Training summary
print("\n" + "=" * 80)
print("Training Summary")
print("=" * 80)

print("\n📊 Generator:")
print(f"  Model: {MODEL_NAME}")
print(f"  Training examples: {len(train_data)}")
print(f"  Validation examples: {len(val_data)}")
print(f"  Final train loss: {train_result.metrics['train_loss']:.4f}")
print(f"  Training time: {train_result.metrics['train_runtime'] / 60:.1f} minutes")
print(f"  Saved to: {GENERATOR_OUTPUT_DIR}")

print("\n📊 Critic:")
print(f"  Model: {MODEL_NAME}")
print(f"  Training examples: {len(train_data)}")
print(f"  Validation examples: {len(val_data)}")
print(f"  Final train loss: {critic_train_result.metrics['train_loss']:.4f}")
print(f"  Training time: {critic_train_result.metrics['train_runtime'] / 60:.1f} minutes")
print(f"  Saved to: {CRITIC_OUTPUT_DIR}")

print("\n" + "=" * 80)
print("Next Steps")
print("=" * 80)
print("\n1. Update model configs:")
print(f"   - configs/generator_config.yaml")
print(f"   - configs/critic_config.yaml")
print("\n2. Run full evaluation:")
print(f"   - Open notebooks/11_generation_evaluation.ipynb")
print(f"   - Run evaluation on 776 queries")
print("\n3. Compare methods:")
print(f"   - No-RAG vs Basic RAG vs Self-RAG vs Self-RAG+INSIDE")
print("\n" + "=" * 80)

In [ ]:
# Save summary to file
summary = {
    'generator': {
        'model': MODEL_NAME,
        'train_examples': len(train_data),
        'val_examples': len(val_data),
        'final_train_loss': float(train_result.metrics['train_loss']),
        'training_time_minutes': float(train_result.metrics['train_runtime'] / 60),
        'output_dir': GENERATOR_OUTPUT_DIR,
    },
    'critic': {
        'model': MODEL_NAME,
        'train_examples': len(train_data),
        'val_examples': len(val_data),
        'final_train_loss': float(critic_train_result.metrics['train_loss']),
        'training_time_minutes': float(critic_train_result.metrics['train_runtime'] / 60),
        'output_dir': CRITIC_OUTPUT_DIR,
    }
}

with open('../results/training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print("✓ Training summary saved to results/training_summary.json")